# Deep Learning Model

# --------------------------------------------------------
## 1) Import packages

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras_tuner.tuners import RandomSearch


# pip install tensorflow
# pip install keras
# pip install keras-tuner

# --------------------------------------------------------
## 2) Load Dataset

In [10]:
df = pd.read_csv("../../data/Student_performance_scaled.csv")
df.head()

,Age,Gender,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GradeClass
0,0.472919,0.978492,0.253711,1.780336,-0.890822,1.522371,-0.108744,-0.788476,-0.660132,2.019544,-0.431866,2.0
1,1.362944,-1.021981,-0.746087,0.997376,-1.717694,-0.656870,-0.999551,-0.788476,-0.660132,-0.495161,-0.431866,1.0
2,-1.307132,-1.021981,1.253509,-0.984045,1.353542,-0.656870,-0.108744,-0.788476,-0.660132,-0.495161,-0.431866,4.0
3,0.472919,0.978492,1.253509,0.045445,-0.063951,-0.656870,0.782063,1.268269,-0.660132,-0.495161,-0.431866,3.0
4,0.472919,0.978492,0.253711,-0.902311,0.290422,1.522371,0.782063,-0.788476,-0.660132,-0.495161,-0.431866,4.0


# --------------------------------------------------------
## 3) Feature Engineering

#### i) Encoding Categorical Variables: 

In [11]:
def encode_categorical_features(df):
    # all features are already scaled, so return unchanged.
    return df

#### ii) Ratio & Aggregate Features: 

adds new features in the for of ratios

`StudyAbsenceRatio` combines `StudyTimeWeekly` and `Absences`. ↑study:↓absent = ↑ratio

In [12]:
# Create ratio-based feature(s)
def add_ratio_features(df):
    df = df.copy()
    # Study Time to Absence ratio
    df['StudyAbsenceRatio'] = df['StudyTimeWeekly'] / (df['Absences'] + 1)  # +1 to avoid division by zero
    return df

#### iii) Interaction Features:

adds new features in terms of interaction

`SportsMusic` multiplies `Sports` and `Music` to give an understanding into the total extra carricular activities a student takes part in

`TotalSupport` adds `TotalSupport` and `Tutoring` to show total support given to a student

In [13]:
#optional
def add_interaction_features(df):
    df = df.copy()
    # Combining sports and music participation
    df['SportsMusic'] = df['Sports'] * df['Music']
    # Combined parental involvement
    df['TotalSupport'] = df['ParentalSupport'] + df['Tutoring']
    return df

#### iV) Apply all feature engineering:

In [14]:
def apply_feature_engineering(df):
    df = encode_categorical_features(df)
    df = add_ratio_features(df)
    df = add_interaction_features(df)
    return df

# --------------------------------------------------------
## 4) Prepare Data

`x` = features (independent variables the model learns from).

`y` = target (GradeClass, the label we want the model to predict).

In [15]:
# Apply feature engineering
df = apply_feature_engineering(df)

# Define features and target
X = df.drop(['GradeClass'], axis=1)
y = df['GradeClass']

# Split the data into train and test sets (ensure y_test is defined)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --------------------------------------------------------
## 5) Build model and set up tuner

using the sequential keras model

Model: https://keras.io/api/models/sequential/

Explained: https://www.geeksforgeeks.org/keras-sequential-class/

In [16]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))
    
    model.add(Dense(units=hp.Int('units2', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(5, activation='softmax'))  # 5 classes for GradeClass

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


keras tuner:

https://keras.io/keras_tuner/api/tuners/random/

automatically searches for the best hyperparameters for the deep learning model instead of using a grid or manual methods.

It works by:
1) Randomly picks different combinations of settings.
2) Trains a model with each.
3) Picks the best based on a metric specified (`objective` = `'val_accuracy'`).

Settings it tries in the script:

`units1`, `units2`: Neurons in 1st and 2nd layers (32 to 256).

`dropout1`, `dropout2`: Dropout rates (0 to 0.5).

In [18]:
# Keras Tuner
tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=10,
    seed=42,
    directory='../../Tuners/student_tuner',
    project_name='grade_classification'
)

tuner.search(X_train, y_train, epochs=20, validation_split=0.2)

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.7467362880706787

Best val_accuracy So Far: 0.7467362880706787
Total elapsed time: 00h 00m 37s


# --------------------------------------------------------
## 6) Choose the best model and run predictions

Best model = the one that scored highest on validation accuracy during tuner search.

Fit the best model again on full training data (20 epochs).

Uuse it to predict the classes for X_test.

In [19]:
# Get best model
best_model = tuner.get_best_models(num_models=1)[0]

# Fit on full training data
best_model.fit(X_train, y_train, epochs=20, validation_split=0.2)

# Predictions
y_pred = np.argmax(best_model.predict(X_test), axis=1)

Epoch 1/20


c:\Program Files\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Program Files\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7336 - loss: 0.8605 - val_accuracy: 0.7285 - val_loss: 0.8514
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7555 - loss: 0.7363 - val_accuracy: 0.7337 - val_loss: 0.8003
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7616 - loss: 0.7472 - val_accuracy: 0.7311 - val_loss: 0.7987
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7668 - loss: 0.6612 - val_accuracy: 0.7337 - val_loss: 0.7846
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7616 - loss: 0.7059 - val_accuracy: 0.7311 - val_loss: 0.7850
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7908 - loss: 0.6596 - val_accuracy: 0.7337 - val_loss: 0.7789
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7764 - loss: 0.6379 - val_accuracy: 0.7258 - val_loss: 0.7909
Epoch 8/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7733 - loss: 0.6396 - val_accuracy: 0.7311 - val_loss: 0.7784
Epo

# --------------------------------------------------------
## 7) Run Evaluation Metrics

#### Accuracy

simple measure of correctness

`correct predictions` / `total predictions`

In [20]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7077244258872651


#### Precision (weighted)

how many predictions were actually correct

weighted adjusts for class imbalance

In [21]:
print("Precision:", precision_score(y_test, y_pred, average='weighted'))

Precision: 0.7210215956962963


#### Recall (weighted)

how many labels were correctly predicted?

weighted adjusts for class imbalance

In [22]:
print("Recall:", recall_score(y_test, y_pred, average='weighted'))

Recall: 0.7077244258872651


#### F1 Score (weighted)

harmonic mean of precision and recall

weighted adjusts for class imbalance

In [23]:
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

F1 Score: 0.7087159825807737


#### Confusion Matrix

shows real vs predicted class counts

In [24]:
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Confusion Matrix:
 [[  4   5   3   5   4]
 [ 12  20  16   4   2]
 [  1   5  52  19   1]
 [  0   0  23  50  10]
 [  1   3   2  24 213]]


#### Classification Report

breakdown of precision, recall, F1-Score per class

In [25]:
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

         0.0       0.22      0.19      0.21        21
         1.0       0.61      0.37      0.46        54
         2.0       0.54      0.67      0.60        78
         3.0       0.49      0.60      0.54        83
         4.0       0.93      0.88      0.90       243

    accuracy                           0.71       479
   macro avg       0.56      0.54      0.54       479
weighted avg       0.72      0.71      0.71       479

